In [10]:
import tensorflow as tf
# dropout layer missing

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist =input_data.read_data_sets("MNIST_data/",one_hot=True)

/Users/shefalibansal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_size_to_hidden= input_width//(max_pool1_k*max_pool2_k) * input_height//(max_pool1_k*max_pool2_k) * n_conv2

In [13]:
weights={
    "wc1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal([input_size_to_hidden , n_hidden  ])),
    "wo":tf.Variable(tf.random_normal([ n_hidden ,n_out ]))
    
}

biases={
    "bc1": tf.Variable(tf.random_normal([n_conv1])),
    "bc2": tf.Variable(tf.random_normal([n_conv2])),
    "bh1": tf.Variable(tf.random_normal([n_hidden])),
    "bo": tf.Variable(tf.random_normal([n_out]))
}


In [15]:
def conv(x,weights,bias,strides=1):
    out= tf.nn.conv2d(x,weights, padding="SAME",strides=[1,strides,strides,1] )
    out= tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

In [16]:
def maxpooling(x,k = 2):
    return tf.nn.max_pool(x,padding="SAME", ksize=[1,k,k,1],strides=[1,k,k,1])

In [17]:
def cnn(x,weights,biases):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    
    hidden_input= tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output= tf.nn.relu(hidden_output_before_activation)
    
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])

    return output

In [18]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_out])
pred=cnn(x,weights,biases)

In [19]:
pred

<tf.Tensor 'Add_1:0' shape=(?, 10) dtype=float32>

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels = y))

In [21]:
cost

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [24]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y})
        total_cost += c
    print(total_cost)

1691673.9738993496
31513.83899628518
16287.12245720589
11943.393375515938
10981.334879021815
10547.214046061039
11241.859083565187
9381.502257318643
9336.563761104673
9116.03763037987
8010.543674198358
7304.329039860691
5783.6504801329975
4502.733837354928
5652.4353271707305
4502.6230967491865
4820.47589390099
4010.2387244838446
3316.1856725472935
3768.3009750969286
3461.556489997651
2625.394039349805
3238.562728743347
3186.7809720311852
2895.7258726536897


In [25]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels})
correct_preds.sum()

9869